<a href="https://colab.research.google.com/github/PhoenixCC0722/Journey_to_become_DataScientist/blob/main/11_bonus_challenges__sql_to_pd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# From SQL to pandas challenge 11

In [2]:
# import libraries
import pandas as pd

# load data
# This code is made to load our data stored on Google Drive
def gd_path(file_id):
    """Generate a shareable link from Google Drive file id."""
    return f"https://drive.google.com/uc?export=download&id={file_id}"

# Google Drive file ids
files_id = {
    "titleauthor": "1F1JOiYXStWacOBca6coNVfyVtoST7ZgD",
    "titles": "1PLdn50N9GRa53ZbuVWo0l47F_IXdvlEm",
    "sales": "1fzFc9rwYmVIPaGOFmhLVxCi3kg19vNU2",
    "roysched": "1zPRZPoFPEMKyrNR5VSENeYFHGCBZmxbs",
    "publishers": "1s9E8_AVOziTrowb3wyh2jg3PV763VOyq",
    "employee": "1h9mUjsVqpP74b1w0x7KOw37n_n9Ulkt5",
    "authors": "1fEF89Nhe61EebAljKlwFwfEuokK0o6aJ"
}

# Read data from Google Drive
sales = pd.read_csv(gd_path(files_id["sales"]), sep=";")
titles = pd.read_csv(gd_path(files_id["titles"]), sep=";")
publishers = pd.read_csv(gd_path(files_id["publishers"]), sep=";")
employee = pd.read_csv(gd_path(files_id["employee"]), sep=";")
authors = pd.read_csv(gd_path(files_id["authors"]), sep=";")
titleauthor = pd.read_csv(gd_path(files_id["titleauthor"]), sep=";")
roysched = pd.read_csv(gd_path(files_id["roysched"]), sep=";")

## 1. Using LEFT JOIN: in which city is the publisher of "Is Anger the Enemy" based?

In [2]:
df_p = pd.DataFrame(publishers)
df_p.head()

,pub_id,pub_name,city,state,country
0,736,New Moon Books,Boston,MA,USA
1,877,Binnet & Hardley,Washington,DC,USA
2,1389,Algodata Infosystems,Berkeley,CA,USA
3,1622,Five Lakes Publishing,Chicago,IL,USA
4,1756,Ramona Publishers,Dallas,TX,USA


In [9]:
df_m = df_p.merge(titles, on='pub_id')
#df_m.head(30)
Pub_city = df_m.loc[(df_m['title'].str.contains('Is Anger the Enemy')),['city']]
print(Pub_city)

     city
1  Boston


### Hint:

In SQL the syntax is:

```sql
SELECT p.city
FROM publishers AS p
LEFT JOIN titles AS t
ON p.pub_id = t.pub_id
WHERE t.title = 'Is Anger the Enemy?';
```

## 2. Select all the book titles that have a link to the employee Howard Snyder
    (he works for the publisher that has published those books).

In [13]:
df_Emp = pd.DataFrame(employee)
df_m = df_Emp.merge(titles,on='pub_id')
#df_m.head()
filt = (df_m['fname']=='Howard') & (df_m['lname']=='Snyder')
df_m.loc[filt,['title']]

,title
135,You Can Combat Computer Stress!
136,Is Anger the Enemy?
137,Life Without Fear
138,Prolonged Data Deprivation: Four Case Studies
139,Emotional Security: A New Algorithm


### Hint:

In SQL the syntax is:

```sql
SELECT t.title
FROM employee e
JOIN titles t
ON e.pub_id = t.pub_id
WHERE e.fname = 'Howard'
AND e.lname = 'Snyder';
```

## 3. Using the `merge` of your choice: Select the book title with highest number of sales (qty)

In [35]:
df_s = pd.DataFrame(sales)
df_m = df_s.merge(titles, on='title_id')
qty_sum = df_m.groupby('title')['qty'].sum()
title_hs = qty_sum.idxmax()
print('Book title with highest number of sales:')
print(title_hs)


Book title with highest number of sales:
Is Anger the Enemy?


### Hint:

In SQL the syntax is:

```sql
SELECT t.title, SUM(qty)
FROM sales AS s
JOIN titles t
ON s.title_id = t.title_id
GROUP BY t.title_id
ORDER BY SUM(qty) desc
LIMIT 1;
```

# 4. Select all book titles and the full name of their author(s).
      
      - If a book has multiple authors, all authors must be displayed (in
      multiple rows).
      
      - Books with no authors and authors with no books should not be displayed.

In [36]:
df_t = pd.DataFrame(titles)
df_t.head()

,title_id,title,type,pub_id,price,advance,royalty,ytd_sales,notes,pubdate
0,BU1032,The Busy Executive's Database Guide,business,1389,19.99,5000.0,10,4095,An overview of available database systems with...,1991-06-12 00:00:00
1,BU1111,Cooking with Computers: Surreptitious Balance ...,business,1389,11.95,5000.0,10,3876,Helpful hints on how to use your electronic re...,1991-06-09 00:00:00
2,BU2075,You Can Combat Computer Stress!,business,736,2.99,10125.0,24,18722,The latest medical and psychological technique...,1991-06-30 00:00:00
3,BU7832,Straight Talk About Computers,business,1389,19.99,5000.0,10,4095,Annotated analysis of what computers can do fo...,1991-06-22 00:00:00
4,MC2222,Silicon Valley Gastronomic Treats,mod_cook,877,19.99,0.0,12,2032,"Favorite recipes for quick, easy, and elegant ...",1991-06-09 00:00:00


In [4]:
df_a = pd.DataFrame(authors)
df_a.head()
df_ta = pd.DataFrame(titleauthor)
df_ta.head()

,au_id,title_id,au_ord,royaltyper
0,172-32-1176,PS3333,1,100
1,213-46-8915,BU1032,2,40
2,213-46-8915,BU2075,1,100
3,238-95-7766,PC1035,1,100
4,267-41-2394,BU1111,2,40


In [6]:
# innerjoin takes the intersection of the two Tables
# result = pd.merge(left, right, how="inner", on=["key1", "key2"])
# NaN should be ignored and not display
Result1 = pd.merge(titles, titleauthor, how="inner", on='title_id')
#Result1.head()
Result2 = pd.merge(Result1, authors , how="inner", on='au_id')
Result2[['title','au_fname','au_lname']]

,title,au_fname,au_lname
0,The Busy Executive's Database Guide,Marjorie,Green
1,You Can Combat Computer Stress!,Marjorie,Green
2,The Busy Executive's Database Guide,Abraham,Bennet
3,Cooking with Computers: Surreptitious Balance ...,Michael,O'Leary
4,"Sushi, Anyone?",Michael,O'Leary
5,Cooking with Computers: Surreptitious Balance ...,Stearns,MacFeather
6,Computer Phobic AND Non-Phobic Individuals: Be...,Stearns,MacFeather
7,Straight Talk About Computers,Dean,Straight
8,Silicon Valley Gastronomic Treats,Innes,del Castillo
9,The Gourmet Microwave,Michel,DeFrance


In [14]:
# Filter out books with no authors and authors with no books
filtered_data = Result2.dropna(subset=['au_id'])

# Display all authors for books with multiple authors in separate rows
result = filtered_data[['title', 'au_fname', 'au_lname']]

# Display the result with all authors for books with multiple authors in separate rows
result

,title,au_fname,au_lname
0,The Busy Executive's Database Guide,Marjorie,Green
1,You Can Combat Computer Stress!,Marjorie,Green
2,The Busy Executive's Database Guide,Abraham,Bennet
3,Cooking with Computers: Surreptitious Balance ...,Michael,O'Leary
4,"Sushi, Anyone?",Michael,O'Leary
5,Cooking with Computers: Surreptitious Balance ...,Stearns,MacFeather
6,Computer Phobic AND Non-Phobic Individuals: Be...,Stearns,MacFeather
7,Straight Talk About Computers,Dean,Straight
8,Silicon Valley Gastronomic Treats,Innes,del Castillo
9,The Gourmet Microwave,Michel,DeFrance


### Hint:

In SQL the syntax is:

```sql
SELECT
    t.title,
    a.au_fname,
    a.au_lname
FROM titles t
INNER JOIN titleauthor ta
ON t.title_id = ta.title_id
INNER JOIN authors a
ON ta.au_id = a.au_id;
```

## 5. Select the full name of authors of Psychology books

   Bonus hint: if you want to prevent duplicates but allow authors with shared
   last names to be displayed, you can concatenate the first and last names
   with CONCAT(), and use the DISTINCT clause on the concatenated names.

In [27]:
# innerjoin
df_merge = pd.merge(authors,titleauthor, on='au_id')
df_merge = pd.merge(df_merge,titles, on='title_id')
# drop_duplicates and concatenated
df_merge = df_merge.drop_duplicates(subset=['au_fname', 'au_lname'])
#df_merge.head()
df_merge = df_merge.loc[(df_merge['type']=='psychology'),['au_fname','au_lname']]
concatenated = pd.concat([df_merge['au_fname'], df_merge['au_lname']], axis=1)
concatenated

,au_fname,au_lname
0,Johnson,White
20,Livia,Karsen
23,Albert,Ringer


### Hint:

In SQL the syntax is:

```sql
SELECT DISTINCT CONCAT(a.au_fname, " ", a.au_lname) AS full_name
FROM authors a
INNER JOIN titleauthor ta ON a.au_id = ta.au_id
INNER JOIN titles t ON ta.title_id = t.title_id
WHERE t.type = "psychology";
```

## 6. Explore the table roysched and try to grasp the meaning of each column.
   The notes below will help:
   
   - "Royalty" means the percentage of the sale price paid to the author(s).
   
   - Sometimes, the royalty may be smaller for the first few sales (which have
     to cover the publishing costs to the publisher) but higher for the sales
     above a certain threshold.
     
   - In the "roysched" table each title_id can appear multiple times, with
     different royalty values for each range of sales.
     
   - Select all rows for particular title_id, for example "BU1111", and explore
	 the data.

In [32]:
df_r = pd.DataFrame(roysched)
df_r.head()
df_r.loc[(df_r['title_id']=='BU1111'),:]

,title_id,lorange,hirange,royalty
49,BU1111,0,4000,10
50,BU1111,4001,8000,12
51,BU1111,8001,10000,14
52,BU1111,12001,16000,16
53,BU1111,16001,20000,18
54,BU1111,20001,24000,20
55,BU1111,24001,28000,22
56,BU1111,28001,50000,24


### Hint:

In SQL the syntax is:

```sql
SELECT * FROM roysched WHERE title_id = "BU1111";
```

## 7. Select all the book titles and the maximum royalty they can reach.
    Display only titles that are present in the roysched table.

In [38]:
#need to return to the original data and remove the former aggregation and then apply the new functions, otherwise the price_avg and price_count can not be found at all
titles = pd.read_csv(gd_path(files_id["titles"]), sep=";")
titles_original = titles.copy() # Make a copy of the original DataFrame before any aggregations
titles = titles_original.copy() # Assuming 'titles_original' is your original DataFrame before any aggregations
titles.head()

roysched = pd.read_csv(gd_path(files_id["roysched"]), sep=";")
roysched_original = roysched.copy() # Make a copy of the original DataFrame before any aggregations
roysched = roysched_original.copy() # Assuming 'titles_original' is your original DataFrame before any aggregations
roysched.head()

df_merge = pd.merge(titles, roysched, how='inner', on='title_id')
df_merge.head() #why here comes the royalty_x and royalty_y????
#df_merge.groupby('title')['royalty'].max(ascending=False)





,title_id,title,type,pub_id,price,advance,royalty_x,ytd_sales,notes,pubdate,lorange,hirange,royalty_y
0,BU1032,The Busy Executive's Database Guide,business,1389,19.99,5000.0,10,4095,An overview of available database systems with...,1991-06-12 00:00:00,0,5000,10
1,BU1032,The Busy Executive's Database Guide,business,1389,19.99,5000.0,10,4095,An overview of available database systems with...,1991-06-12 00:00:00,5001,50000,12
2,BU1111,Cooking with Computers: Surreptitious Balance ...,business,1389,11.95,5000.0,10,3876,Helpful hints on how to use your electronic re...,1991-06-09 00:00:00,0,4000,10
3,BU1111,Cooking with Computers: Surreptitious Balance ...,business,1389,11.95,5000.0,10,3876,Helpful hints on how to use your electronic re...,1991-06-09 00:00:00,4001,8000,12
4,BU1111,Cooking with Computers: Surreptitious Balance ...,business,1389,11.95,5000.0,10,3876,Helpful hints on how to use your electronic re...,1991-06-09 00:00:00,8001,10000,14


### Hint:

In SQL the syntax is:

```sql
SELECT t.title, MAX(r.royalty) max_royalty
FROM titles t
INNER JOIN roysched r
ON t.title_id = r.title_id
GROUP BY t.title
ORDER BY max_royalty DESC;
```